In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm 
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
from scipy import stats
import warnings

with warnings.catch_warnings():
    warnings.simplefilter("ignore")

In [ ]:
# function for calculating pdf

def conditional_densities(data,formula_ps_no_con,formula_ps_con, use_confounders=True):
    
    formula = formula_ps_con if use_confounders else formula_ps_no_con
        
    model = sm.formula.ols(formula, data=data).fit()
    density = stats.norm( 
        loc=model.fittedvalues,
        scale=model.resid.std(),
    )  
    densities = density.pdf(data["nSES"])
    densities = pd.Series(densities, index=model.fittedvalues.index)
    return densities


## data

In [ ]:
mesa_std = pd.read_csv('../data_processed/MESA/mesa_std.csv', index_col=False)
mesa_std = mesa_std.dropna()

analysis_feature = ['cvd_10y_HF', 'cvd_10y_noHF',
                    'nSES','nFavFood','nPhysFac', 'nRS', 
                    'FamIncome', 'nutrition', 'PhysAct', 'currentSmoker', 'alc',
                    'age', 'gender', 'Diabetes', 'hdl', 'totchol', 'sbp',
                   'site', 'race']

mesa_std = mesa_std[analysis_feature]

mesa_bla_std = mesa_std[mesa_std['race'] == 1]


In [ ]:
jhs_std = pd.read_csv('../data_processed/JHS/jhs_std.csv', index_col=False)
jhs_std = jhs_std.dropna()

analysis_feature = ['cvd_10y_HF', 'cvd_10y_noHF',
                    'nSES','nFavFood','nPhysFac', 'nRS', 
                    'FamIncome', 'nutrition', 'PhysAct', 'currentSmoker', 'alc',
                    'age','gender', 'Diabetes', 'hdl', 'totchol', 'sbp']

jhs_std = jhs_std[analysis_feature]

In [ ]:
# check nSES distribution

# jhs_std['nSES'].plot.kde()

# plt.title('Density Plot for nSES')
# plt.xlabel('nSES Values')
# plt.ylabel('Density')

# plt.show()

## Generalized IPW for single estimate

In [ ]:
# Step 1: estimate propensity density

denominator = conditional_densities(data, formula_ps_no_con,formula_ps_con, use_confounders=True)
numerator = conditional_densities(data, formula_ps_no_con,formula_ps_con, use_confounders=False)
generalized_ipw = numerator / denominator

In [ ]:
# check balance??

# love plot displays standardized mean differences, but only works for binary treatment


In [ ]:
# Step 2: Weighted Logistic regression 

formula_outcome = 'Y_10y ~ 1 + nSES'

model = smf.logit(formula=formula_outcome, data=data, weights=generalized_ipw).fit()
print(model.summary())

In [ ]:
# visualize the predictions

dosage = list(range(-10,10))  # hypothesized dosage value for plotting
dosage = pd.DataFrame(
    data={"nSES": dosage},
    index=dosage,
)
response = model.predict(dosage)
ax = response.plot(
    kind="line",
    xlabel="nSES value",
    ylabel="Prediction of 10-year CVD risk"
)


In [ ]:
# check logistic regression assumption 

import pygam
from pygam import LogisticGAM, s

X = data['nSES']  # Predictor variables
y = data['Y_10y']  # Response variable

# Fit a GAM with a spline for each predictor variable
gam = LogisticGAM(s(0)).fit(X, y)

# Plot the partial dependence for each predictor to inspect linearity
for i, term in enumerate(gam.terms):
    if term.isintercept:
        continue
    XX = gam.generate_X_grid(term=i)
    pdep, confi = gam.partial_dependence(term=i, X=XX, width=0.95)
    plt.figure()
    plt.plot(XX[:, i], pdep)
    plt.plot(XX[:, i], confi, c='r', ls='--')
    plt.title(f'Partial Dependence')
    plt.show()

## roughly linear, assumption holds

### bootstrap for CI

In [ ]:
def generate_formula(X_vars, M_vars,additional_vars, data):
    
    # Define different settings based on the dataset
    if data == "mesa":
        settings = [
            additional_vars + ['race', 'site'],
            additional_vars + ['race'],
            additional_vars + ['site'],
            additional_vars
        ]
    elif data == "mesa_bla":
        settings = [
            additional_vars + ['site'],
            additional_vars
        ]
    elif data == "jhs":
        settings = [additional_vars]  # No 'race' or 'site'
    else:
        settings = [additional_vars]  # Default setting, can be modified as needed
    
    formula_combinations = []

    # Generate combinations
    for setting in settings:
        
        Z = set(setting)  # Adjust Z based on the current setting
        
        for i in range(len(X_vars)):
        
            # Rotate X variables so a different one is in the first position
            rotated_X_vars = X_vars[i:] + X_vars[:i]
            
            X = rotated_X_vars[0]

            # Create the three formulas based on the current rotation
            formula_ps_no_con = f"{rotated_X_vars[0]} ~ 1"
            formula_ps_con = f"{rotated_X_vars[0]} ~ 1 + {' + '.join(rotated_X_vars[1:])} + " + \
                             " + ".join([f"C({m})" for m in M_vars]) + " + " + \
                             " + ".join(Z)
            formula_outcome = f'cvd_10y_noHF ~ 1 + {rotated_X_vars[0]}'   ###### noHF Y

            # Add the set of formulas to the list
            formula_combinations.append({
                'X': [X],
                'Z': list(Z),
                'formula_ps_no_con': formula_ps_no_con,
                'formula_ps_con': formula_ps_con,
                'formula_outcome': formula_outcome
            })
            
    return formula_combinations

In [ ]:
def case_when(Z):
    if 'race' in Z and 'site' in Z:
        return 'race_site'
    elif 'race' in Z:
        return 'race'
    elif 'site' in Z:
        return 'site'
    else:
        return None  # Or any default value

In [ ]:
# generate combination

# Define the variables
X_vars = ['nSES', 'nFavFood', 'nPhysFac', 'nRS']
M_vars = ['FamIncome', 'nutrition', 'PhysAct', 'currentSmoker', 'alc']
additional_vars = ['age', 'sbp', 'hdl', 'totchol', 'Diabetes']

# define the data
named_df_list = [('mesa_std', mesa_std)]
data_option = "mesa" 

# generate combo
formula_combinations = generate_formula(X_vars, M_vars,additional_vars, data_option)

In [ ]:
report_df = pd.DataFrame(columns=['Data','X','other','mean_coef', 'ci_lower','ci_upper' ])
iteration = 1000

for df_name, df in named_df_list:
    
    ### iterate on each formula combo ###
    for combo in formula_combinations:
        X = combo['X']
        Z = combo['Z']
        formula_ps_no_con = combo['formula_ps_no_con'] 
        formula_ps_con = combo['formula_ps_con'] 
        formula_outcome = combo['formula_outcome']
        
        coef_treatment = []
        
        for i in range(iteration):
            d_star = df.sample(n=df.shape[0], replace=True)

            # step 1
            denominator = conditional_densities(d_star, formula_ps_no_con,formula_ps_con, use_confounders=True)
            numerator = conditional_densities(d_star, formula_ps_no_con,formula_ps_con, use_confounders=False)
            generalized_ipw = numerator / denominator

            # step 2
            with warnings.catch_warnings():
                warnings.simplefilter("ignore", category=UserWarning)
            model = smf.logit(formula=formula_outcome, data=d_star, weights=generalized_ipw).fit(disp=0)
            coef_treatment.append(model.params[1])

        # Calculate mean and 95% CI
        mean_coef = np.round(np.mean(coef_treatment), 5)
        ci_lower = np.round(np.percentile(coef_treatment, q=2.5), 5)
        ci_upper = np.round(np.percentile(coef_treatment, q=97.5), 5)

        new_df = pd.DataFrame(
                    {'Data': df_name,
                     'X': X,
                     'other': case_when(Z),
                     'mean_coef':mean_coef,
                     'ci_lower': ci_lower,
                     'ci_upper':ci_upper
                    })

        # append to the complete dataset
        report_df = pd.concat([report_df, new_df], ignore_index=True)
            
        # print(report_df)
        print("\n" + "="*50 + "\n")  


export_path = f"../results/G_IPW_noHF/{df_name}_G_ipw.csv"
report_df.to_csv(export_path)